# Style-based quantum generative adversarial networks for Monte Carlo events

Code at:

Paper:

In this tutorial, we show how to implement a style-based quantum generative adversarial network (style-qGAN). We provide the tools for training the style-qGAN on a reference example, namely, a 3D correlated Gaussian distribution. For this particular implementation, we are going to use the Qibo API for the simulation of quantum circuits, and the Keras packages for the execution of classical neural networks.

## Framework

The generative adversarial framework is a powerful tool for training generator models. It involves at least three required components: the discriminator model, the generator model, and the adversarial training procedure. In this tutorial, we consider a hybrid quantum-classical system, where the generator model has a quantum representation through a quantum neural network, while the discriminator is a classical neural network model.

As we show in the figure below, the procedure starts from the preparation of samples from a known distribution function that we would like to encode in the quantum generator model. At the same time, we define a quantum neural network model where we inject stochastic noise in the latent space variables which are used to define quantum gates. The generator model is then used to extract fake samples that after the training procedure should match the quality of the known input distribution. Both sets of samples are then used to train the discriminator model. The quality of the training is measured by an appropriate loss function which is monitored and optimized classically by a minimization algorithm based on the adversarial approach. The training process consists of a simultaneous stochastic gradient descent for both models which after reaching convergence delivers a quantum generator model with realistic sampling.

<img src="scheme.png" width="320px">

This is a complex type of model both to understand and to train. A possible approach to better understand the nature of this style-qGAN model and how it can be trained is to develop the model from scratch.

In this tutorial, we will select a simple distribution and use it as the basis for developing and evaluating a quantum generative adversarial network from scratch: a 3D correlated Gaussian distribution

## Refererence model

The starting point should be generating real samples for our discriminator. Specifically, we will generate real samples from a 3D correlated Gaussian distribution. A sample is going to be comprised of a vector with three elements, one for each dimension. We will normalize this samples by dividing by 4, so each sample is within the [-1,1] range.

In [1]:
# imports
import numpy as np
from numpy.random import randn

def generate_training_real_samples(samples):
  # generate training samples from the distribution
    s = []
    mean = [0, 0, 0]
    cov = [[0.5, 0.1, 0.25], [0.1, 0.5, 0.1], [0.25, 0.1, 0.5]]
    x, y, z = np.random.multivariate_normal(mean, cov, samples).T/4
    s1 = np.reshape(x, (samples,1))
    s2 = np.reshape(y, (samples,1))
    s3 = np.reshape(z, (samples,1))
    s = np.hstack((s1,s2,s3))
    return s

## Define a discriminator model

The discriminator model should take a sample from our problem, in this case a vector with three elements, and output a prediction as to whether the samples are fake or real. In this case, we will choose a deep convolutional neural network, with several hidden layers and we will use the ReLU activation function. The output layer will have one node for the binary classification using the sigmoid activation function. The model will minimize the binary cross-entropy loss function, and the we will use the stochastic gradient descent method Adadelta to update the parameters.

The model must take a sample from our problem, such as a vector with two elements, and output a classification prediction as to whether the sample is real or fake. This is done by the function `define_discriminator()` below.

We will also implement a function that will be useful later on for the batch optimization. This function is `generate_real_samples()`, which from a particular set of samples, it chooses a batch of some particular size and gives the class label (real in this case).

In [2]:
# imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Reshape, LeakyReLU, Flatten

# define the standalone discriminator model
def define_discriminator(n_inputs=3, alpha=0.2, dropout=0.2):
    model = Sequential()
        
    model.add(Dense(200, use_bias=False, input_dim=n_inputs))
    model.add(Reshape((10,10,2)))
    
    model.add(Conv2D(64, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_normal'))
    model.add(LeakyReLU(alpha=alpha))
    
    model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_normal'))
    model.add(LeakyReLU(alpha=alpha))

    model.add(Conv2D(16, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_normal'))
    model.add(LeakyReLU(alpha=alpha))

    model.add(Conv2D(8, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_normal'))

    model.add(Flatten())
    model.add(LeakyReLU(alpha=alpha))
    model.add(Dropout(dropout)) 

    model.add(Dense(1, activation='sigmoid'))
    
    # compile model
    opt = Adadelta(learning_rate=0.1)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create classical discriminator
discriminator = define_discriminator()

# generate real samples with class labels
def generate_real_samples(samples, distribution, real_samples):
    # generate samples from the distribution
    idx = np.random.randint(real_samples, size=samples)
    X = distribution[idx,:]
    # generate class labels
    y = np.ones((samples, 1))
    return X, y

## Define a quantum generator model

Before defining the generator model, we first should provide with some of the tools needed to create it. Specifically, we should provide the number of qubits (which is 3), and what expectation values do we want to generate the fake samples (`hamiltonian1()`, `hamiltonian2()`, `hamiltonian3()`) that will act on each of the qubits.

In [3]:
# imports
import tensorflow as tf
from qibo import gates, hamiltonians, models, set_backend

# set qibo backend
set_backend('tensorflow')

# number of qubits generator
nqubits = 3

# define hamiltonian to generate fake samples
def hamiltonian1():
    id = [[1, 0], [0, 1]]
    m0 = hamiltonians.Z(1, numpy=True).matrix
    m0 = np.kron(id, np.kron(id, m0))
    ham = hamiltonians.Hamiltonian(3, m0)
    return ham

def hamiltonian2():
    id = [[1, 0], [0, 1]]
    m0 = hamiltonians.Z(1, numpy=True).matrix
    m0 = np.kron(id, np.kron(m0, id))
    ham = hamiltonians.Hamiltonian(3, m0)
    return ham

def hamiltonian3():
    id = [[1, 0], [0, 1]]
    m0 = hamiltonians.Z(1, numpy=True).matrix
    m0 = np.kron(m0, np.kron(id, id))
    ham = hamiltonians.Hamiltonian(3, m0)
    return ham

# create hamiltonians
hamiltonian1 = hamiltonian1()
hamiltonian2 = hamiltonian2()
hamiltonian3 = hamiltonian3()

The quantum generator model will take as input a set of latent variables and generate a new sample, in this case, a vector with three components. Latent variables are variables that are not directly observed but will be rather inferred by the quantum generator model. This is because the latent space has no meaning until the quantum generator model starts assigning meaning to points in the space as it learns. We will define a latent space of three dimensions and use the standard approach from the classical GAN literature by using a standard Gaussian distribution for each variable in the latent space. This is done by the function `generate_latent_points()`. Then, we will generate new fake samples by implementing our quantum circuit (see Figure below) with some particular number of layers, and using the function `generate_fake_samples()`.

<img src="ansatz.png" width="550px">

The latent variables $\xi$ and trainable parameters $\phi_g$ are going be to encoded inside quantum rotation gate as

<img src="encoding.png" width="350px">

In [4]:
# number of layers
layers = 1

# create quantum generator
circuit = models.Circuit(nqubits)
for l in range(layers):
    for q in range(nqubits):
        circuit.add(gates.RY(q, 0))
        circuit.add(gates.RZ(q, 0))
        circuit.add(gates.RY(q, 0))
        circuit.add(gates.RZ(q, 0))
    if l==0 or l==2 or l==4 or l==6 or l==8:
        circuit.add(gates.CRY(0, 1, 0))
        circuit.add(gates.CRY(0, 2, 0))
    if l==1 or l==3 or l==5 or l==7 or l==9:
        circuit.add(gates.CRY(1, 2, 0))
        circuit.add(gates.CRY(2, 0, 0))
for q in range(nqubits):
    circuit.add(gates.RY(q, 0))
    
def set_params(circuit, params, x_input, i, nqubits, layers, latent_dim):
    p = []
    index = 0
    noise = 0
    for l in range(layers):
        for q in range(nqubits):
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
        if l==0 or l==2 or l==4 or l==6 or l==8:
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
        if l==1 or l==3 or l==5 or l==7 or l==9:
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
            p.append(params[index]*x_input[noise][i] + params[index+1])
            index+=2
            noise=(noise+1)%latent_dim
    for q in range(nqubits):
        p.append(params[index]*x_input[noise][i] + params[index+1])
        index+=2
        noise=(noise+1)%latent_dim
    circuit.set_parameters(p) 

In [5]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, samples):
    # generate points in the latent space
    x_input = randn(latent_dim * samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(samples, latent_dim)
    return x_input

In [6]:
# use the generator to generate fake examples, with class labels
def generate_fake_samples(params, latent_dim, samples, circuit, nqubits, layers, hamiltonian1, hamiltonian2, hamiltonian3):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, samples)
    x_input = np.transpose(x_input)
    # generator outputs
    X1 = []
    X2 = []
    X3 = []
    # quantum generator circuit
    for i in range(samples):
        set_params(circuit, params, x_input, i, nqubits, layers, latent_dim)
        circuit_execute = circuit.execute()
        X1.append(hamiltonian1.expectation(circuit_execute))
        X2.append(hamiltonian2.expectation(circuit_execute))
        X3.append(hamiltonian3.expectation(circuit_execute))
    # shape array
    X = tf.stack((X1, X2, X3), axis=1)
    # create class labels
    y = np.zeros((samples, 1))
    return X, y

## Training the adversarial model

Here, the trainable parameters of the quantum generator model and classical discriminator model are going to be updated based on the performance of each other. This defines a zero-sum adversarial game between the models. The simplest approach is to create a new function that encapsulates the generation of fake samples and the evaluation of the discriminator model. This function is `define_cost_gan()`. The generator will receive as input random points in the latent space, generate fake samples and then fed them into the discriminator model. The output will be used to update the trainable parameters of the generator.

The discriminator is concerned with distinguishing the real and fake samples, and therefore, can be trained in a standalone manner. Both the training of the discriminator and quantum generator is done in the function `train()`. Once both models are trained, we obtain a quantum generator capable of reproducing the reference distribution.

In [7]:
# define the combined generator and discriminator model, for updating the generator
def define_cost_gan(params, discriminator, latent_dim, samples, circuit, nqubits, layers, hamiltonian1, hamiltonian2, hamiltonian3):
    # generate fake samples
    x_fake, y_fake = generate_fake_samples(params, latent_dim, samples, circuit, nqubits, layers, hamiltonian1, hamiltonian2, hamiltonian3)
    # create inverted labels for the fake samples
    y_fake = np.ones((samples, 1))
    # evaluate discriminator on fake examples
    disc_output = discriminator(x_fake)
    loss = tf.keras.losses.binary_crossentropy(y_fake, disc_output)
    loss = tf.reduce_mean(loss)
    return loss

In [8]:
# train the generator and discriminator
def train(d_model, latent_dim, layers, nqubits, training_samples, discriminator, circuit, n_epochs, samples, lr, hamiltonian1, hamiltonian2, hamiltonian3):
    d_loss = []
    g_loss = []
    # determine half the size of one batch, for updating the discriminator
    half_samples = int(samples / 2)
    initial_params = tf.Variable(np.random.uniform(-0.15, 0.15, 8*layers*nqubits + 4*layers + 2*nqubits))
    optimizer = tf.optimizers.Adadelta(learning_rate=lr)
    # prepare real samples
    s = generate_training_real_samples(training_samples)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_samples, s, training_samples)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(initial_params, latent_dim, half_samples, circuit, nqubits, layers, hamiltonian1, hamiltonian2, hamiltonian3)
        # update discriminator
        d_loss_real, _ = d_model.train_on_batch(x_real, y_real)
        d_loss_fake, _ = d_model.train_on_batch(x_fake, y_fake)
        d_loss.append((d_loss_real + d_loss_fake)/2)
        # update generator
        with tf.GradientTape() as tape:
            loss = define_cost_gan(initial_params, d_model, latent_dim, samples, circuit, nqubits, layers, hamiltonian1, hamiltonian2, hamiltonian3)
        grads = tape.gradient(loss, initial_params)
        optimizer.apply_gradients([(grads, initial_params)])
        g_loss.append(loss)
        np.savetxt(f"PARAMS_3Dgaussian_{nqubits}_{latent_dim}_{layers}_{training_samples}_{samples}_{lr}", [initial_params.numpy()], newline='')
        np.savetxt(f"dloss_3Dgaussian_{nqubits}_{latent_dim}_{layers}_{training_samples}_{samples}_{lr}", [d_loss], newline='')
        np.savetxt(f"gloss_3Dgaussian_{nqubits}_{latent_dim}_{layers}_{training_samples}_{samples}_{lr}", [g_loss], newline='')
        # serialize weights to HDF5
        discriminator.save_weights(f"discriminator_3Dgaussian_{nqubits}_{latent_dim}_{layers}_{training_samples}_{samples}_{lr}.h5")


Here we can select the training setup and start the training.

In [ ]:
# training setup
latent_dim = 3
training_samples = 10000
batch_samples = 128
n_epochs = 30000
lr = 0.5

# train model
train(discriminator, latent_dim, layers, nqubits, training_samples, discriminator, circuit, n_epochs, batch_samples, lr, hamiltonian1, hamiltonian2, hamiltonian3)